In [20]:
import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [72]:
# RNN 기본 구조 
# hihello -> ihello

# data
# vocabulary
idx2char = ['h','i','e','l','o']
xdata = [[0,1,0,2,3,3]] # h i h e l l 
xOneHot = [[
            [1,0,0,0,0], # h
            [0,1,0,0,0], # i 
            [1,0,0,0,0], # h
            [0,0,1,0,0], # e
            [0,0,0,1,0], # l
            [0,0,0,1,0]  # l
]]
#            ihello
ydata = [[1,0,2,3,3,4]]

# 변수 정의
num_classes = 5
input_dim = 5  # 원핫의 크기 = xdata 수
hidden_size = 5 # RNN (LSTM)에서 출력의 크기
batch_size = 1 # 문장 1개
sequence_length = 6
learning_rate = 0.1

In [73]:
tf.reset_default_graph()

# model 정의
x = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
y = tf.placeholder(tf.int32, [None, sequence_length])

cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size,tf.float32)
outputs , _states = tf.nn.dynamic_rnn(cell, x, initial_state=initial_state, dtype=tf.float32)

In [74]:
xForFc = tf.reshape(outputs, [-1,hidden_size])

outputs = tf.contrib.layers.fully_connected(
    inputs = xForFc, num_outputs=num_classes, activation_fn=None
)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

In [75]:
# weight 를 1,1,1,1,1,1 로 초기화
weights = tf.ones([batch_size, sequence_length])

In [76]:
# cost값 구하기
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=y, weights=weights
)

loss = tf.reduce_mean(sequence_loss)

In [77]:
# opt 지정
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [78]:
# 출력노드
prediction = tf.argmax(outputs, axis=2)

In [84]:
# 학습
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    for step in range(100) :
        lossv, pred, _ = sess.run([loss, prediction, train], feed_dict={x:xOneHot, y:ydata})
        
        print(step,'\ncost : ',lossv,'prediction : ',pred, 'true Y',ydata)
        
        s = [idx2char[c] for c in np.squeeze(pred)]
        print('prediction str : ',''.join(s),'\n')

0 
cost :  1.6322551 prediction :  [[0 4 0 1 1 1]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  hohiii 

1 
cost :  1.508181 prediction :  [[3 3 3 3 3 3]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  llllll 

2 
cost :  1.4189187 prediction :  [[3 3 3 3 3 3]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  llllll 

3 
cost :  1.3380836 prediction :  [[3 3 3 3 3 3]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  llllll 

4 
cost :  1.2272444 prediction :  [[3 3 3 3 3 3]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  llllll 

5 
cost :  1.0780529 prediction :  [[3 0 3 3 3 3]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  lhllll 

6 
cost :  0.91664714 prediction :  [[0 0 2 3 3 3]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  hhelll 

7 
cost :  0.774923 prediction :  [[0 0 2 3 3 3]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  hhelll 

8 
cost :  0.65646416 prediction :  [[0 0 2 3 3 4]] true Y [[1, 0, 2, 3, 3, 4]]
prediction str :  hhello 

9 
cost :  0.549041 prediction :  [[1 0 2 3 3 

In [86]:
np.squeeze([[1, 0, 2, 3, 3, 4]])

array([1, 0, 2, 3, 3, 4])

In [87]:
x = np.array([1,2])
x.shape  # (2,)

y = np.expand_dims(x, axis=0)
y.shape  # (1, 2)
print(y)
print(np.expand_dims(x, axis=1))

[[1 2]]
[[1]
 [2]]


In [90]:
x = np.array([[[0],[1],[2]]])
print(x.shape)
print(x)
np.squeeze(x)

(1, 3, 1)
[[[0]
  [1]
  [2]]]


array([0, 1, 2])

In [110]:
# tf 세선 초기화
tf.reset_default_graph()

In [111]:
# 입력이 4인 data
xdata = np.array([[[1,0,0,0]]], dtype=np.float32)
xdata.shape

(1, 1, 4)

In [112]:
# RNN 셀에서의 출력 개수
hidden_size = 2

# LSTM 셀 생성
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
# (1,1,4) -> (1,1,2)

In [117]:
with tf.Session() as sess :
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    print(outputs.eval())

[[[ 0.12422247 -0.02433523]]]


In [125]:
# tf 세선 초기화
tf.reset_default_graph()

# h -> [1,0,0,0]
# e -> [0,1,0,0]
# l -> [0,0,1,0]
# o -> [0,0,0,1]

# 입력이 4인 data
xdata = np.array([[[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,1,0], [0,0,0,1]],
                   [[0,1,0,0], [0,0,0,1], [0,0,1,0], [0,0,1,0], [0,0,1,0]],
                   [[0,0,1,0], [0,0,1,0], [0,1,0,0], [0,1,0,0], [0,0,1,0]]], dtype=np.float32)
xdata.shape

# RNN 셀에서의 출력 개수
hidden_size = 2

# LSTM 셀 생성
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
# (3,5,4) -> (3,5,2)

with tf.Session() as sess :
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    
    sess.run(tf.global_variables_initializer())
    print(outputs.eval())

[[[-0.12443303 -0.06212903]
  [-0.06276275 -0.00828813]
  [-0.1636521  -0.08893296]
  [-0.23261312 -0.1442564 ]
  [-0.10857309 -0.16634823]]

 [[ 0.0153278   0.05680929]
  [ 0.09889258  0.04175255]
  [-0.00271664 -0.05954198]
  [-0.11044564 -0.12421316]
  [-0.19782348 -0.1658223 ]]

 [[-0.10229745 -0.08410886]
  [-0.18854892 -0.14083152]
  [-0.13287292 -0.1459248 ]
  [-0.11592653 -0.08282536]
  [-0.2175776  -0.12214737]]]


In [126]:
# MNIST RNN 

In [131]:
# mnist 데이터 불러오기
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./mnist/data/', one_hot=True)

In [173]:
# 변수 설정
lr = 0.01
total_epoch = 5
batch_size = 128  # 128개의 데이터를 보고 가중치를 업데이트 하겠다.

n_input = 28   # 한 번에 입력받는 데이터 개수
n_step = 28    # 28줄
n_hidden =128  # 셀에서 나가는 출력의 개수
n_class = 10   # 숫자의 종류 (0~9)

In [174]:
tf.reset_default_graph()

In [175]:
#                                        28       28
x = tf.placeholder(tf.float32, [None, n_step, n_input])
#                                        10
y = tf.placeholder(tf.float32, [None, n_class])


w = tf.Variable(tf.random_normal([n_hidden, n_class], dtype=tf.float32))
b = tf.Variable(tf.random_normal([n_class], dtype=tf.float32))

In [176]:
# rnn 학습 셀 생성
cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=n_hidden)

# rnn 망 구성
outputs , _states = tf.nn.dynamic_rnn(cell=cell, inputs=x, dtype=tf.float32)
# outputs : [batch_size, n_step, n_hidden]
# =>  구조변경 (y구조로) => y:[batch_size, n_class]

#      0         1        2              1         0          2
# [batch_size, n_step, n_hidden] - > [ n_step, batch_size, n_hidden]
outputs = tf.transpose(outputs, [1,0,2])
# n_step을 제거
outputs = outputs[-1]


In [177]:
model = tf.matmul(outputs, w) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=y))

opt = tf.train.AdamOptimizer(lr).minimize(cost)

In [178]:
isCorrect = tf.equal(tf.argmax(model, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(isCorrect, tf.float32))
testSize = len(mnist.test.images)

testX = mnist.test.images.reshape(testSize, n_step, n_input)
testY = mnist.test.labels

In [179]:
total_batch = int(mnist.train.num_examples / batch_size)
# 모델 학습
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(total_epoch) :
        total_cost = 0
        for i in range(total_batch) :
            batchX, batchY = mnist.train.next_batch(batch_size)
            batchX = batchX.reshape((batch_size, n_step, n_input))  # 128,784 -> 128,28,28
            
            _, costv = sess.run([opt, cost], feed_dict={x:batchX, y:batchY})
            
            total_cost += costv
        print('에폭 : %4d' %(epoch+1),
             'cost 평균 : {:.3f}'.format(total_cost/total_batch))
    


    print(sess.run(accuracy, feed_dict={x:testX, y:testY}))

에폭 :    1 cost 평균 : 0.351
에폭 :    2 cost 평균 : 0.087
에폭 :    3 cost 평균 : 0.065
에폭 :    4 cost 평균 : 0.055
에폭 :    5 cost 평균 : 0.049
0.9854
